# Segmentation

## Descrizione dell'algoritmo utilizzato

### Input e output

L'input è composto da:
- il documento, cioè una singola stringa composta da frasi concatenate senza alcun capoverso (per la costruzione esatta si rimanda a `build_input_file.ipynb`)
- il numero di paragrafi che vogliamo individuare nel documento

L'output dell'algoritmo è una piccola lista contenete le posizioni dei tagli tra un paragrafo e l'altro.

### Iperparametri

Ci sono due iperparametri il cui scopo verrà chiarito commentando direttamente il codice. Essi sono:
- numero di parole più frequenti;
- span per l'estrazione dei risultati.

### Ipotesi fatte

L'algoritmo che ho implementato si basa su alcune ipotesi:
1. Ogni paragrafo avrà un suo argomento, che sarà sufficientemente diverso dall'argomento degli altri paragrafi. In seguito verrà chiarito cosa si intende esattamente con "sufficientemente diverso". 
2. Si possono utilizzare le parole più frequenti (parole-argomento) per individuare gli argomenti di ciascun paragrafo. In particolare, quindi, esiste una corrispondenza tra i paragrafi e i sottoinsiemi di una specifica partizione dell'insieme delle parole più frequenti. 
3. Si può utilizzare la presenza o l'assenza delle parole-argomento in una frase per determinare a quale argomento (e quindi paragrafo) appartiene quella frase. In particolare le frasi che contengono parole-argomento saranno dette "immediatamente classificabili"; viceversa quelle che non ne contengono alcuna saranno dette "non immediatamente classificabili" o, per brevità "non classificabili".
4. Una frase non immediatamente classificabile, apparterrà al paragrafo "più vicino", cioè al paragrafo di appartenenza della frase immediatamente classificabile più vicina.

L'ipotesi 4 è senza dubbio la più forte tra quelle fatte, e ha un effetto molto importante anche sulla correttezza dell'algoritmo: difficilmente questo algoritmo restituirà la posizione esatta dei tagli tra un paragrafo e l'altro, ma ne riuscità a dare la posizione approssimativa. Questa posizione non mi aspetto che sia troppo lontana da quella effettiva perché l'ipotesi 4 è vera se ci troviamo all'interno di un paragrafo e non sui bordi. Pertanto allontanadosi dai tagli effettivi l'applicazione dell'ipotesi 4 non introduce alcuna approssimazione.

### Spiegazione dell'algoritmo

L'algoritmo si divide in più fasi:
1. Individuazione degli argomenti di cui si parla nel documento di input sfruttando le ipotesi 1 e 2. Questa fase si divide a sua volta in:
   1.  Individuazione delle parole-argomento;
   2.  Ottenimento degli embedding di tali parole;
   3.  Clustering con KMeans sugli embedding delle parole-argomento;
2. Classificazione delle frasi del documento basandosi su ipotesi 3 e 4. Avremo quindi rispettivamente:
   1. Classificazione per presenza: per ogni frase viene calcolata la sua sovrapposizione con le parole-argomento di ogni cluster. La frase viene quindi classificata con il cluster che massimizza la sovrapposizione.
   2. Classificazione per prossimità: per ogni frase non immediatamente classificabile, le si assegna il numero di paragrafo della frase immediatamente classificabile più vicina.
3. Estrazione dei tagli tramite una specie di derivata. Per una spiegazione dettagliata si rimanda all'implementazione.

## Inizializzazione

### Import

In [132]:
from gensim.test.utils import simple_preprocess
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from collections import Counter
from statistics import mean
import json
import pprint

### Recupero del documento di input

Come accennato prima, la costruzione di questo documento avviene in `buil_input_file.ipynb`. Sono state prese tre pagine di Wikipedia (Film, Therapy, Napeoleon Bonaparte) da cui sono stati estratti 5 paragrafi, con frasi di più di 100 caratteri l'uno, che sono stati accorpati in un'unica riga. Sono state anche salvate le posizioni (in termini di frasi) dei tagli corretti.

In [148]:
def retrieve_data(path):
    data = ''
    correct_cuts = []
    with open(path, 'r', encoding='utf8') as f:
        data = f.readline().strip('\n')
        correct_cuts = [int(x) for x in f.readline().strip('\n').split(' ')]
    return data.split('.'), correct_cuts

input_data, correct_cuts = retrieve_data('resources/input_data.txt')
print(f'Numero di frasi nel documento: {len(input_data)}')

Numero di frasi nel documento: 79


### Recupero degli embeddings

Gli embedding salvati nel file `resources/embeddings.json` sono stati estratti dal modello `'fasttext-wiki-news-subwords-300'` di gensim. L'estrazione avviene sempre in `buil_input_file.ipynb`. 

In [135]:
def in_model(word):
    try:
        word_embeddings_model[word]
    except KeyError:
        return False
    return True

def get_embeddings(path):
    embeddings = []
    with open(path, 'r', encoding='utf8') as f:
        embeddings = json.load(f)
    return embeddings

word_embeddings_model = get_embeddings('resources/embeddings.json')

## Esecuzione

### 1.1 Individuazione parole-argomento

Qui introduciamo il primo iperparametro: il numero delle parole più frequenti. Io ho scelto 9 in quanto questo permette intuitivamente di avere 3 parole-argomento per ogni paragrafo.

In [136]:
def build_freq_dict(input_doc):
    frequency_dict = {}
    prep_input_doc = simple_preprocess(". ".join(input_doc))
    for word in prep_input_doc:
        if word not in stopwords.words("english"):
            if is_plural(word):
                word = to_singular(word)
            if word not in frequency_dict.keys():
                frequency_dict[word] = 0
            frequency_dict[word] += 1
    return frequency_dict

def is_plural(word):
    return word[-1] == 's' or word[-2:] == 'es'

def to_singular(word):
    if word[-2:] == 'es':
        return word[:-2]
    if  word[-1] == 's':
        return word[:-1]
    return word

def get_most_frequent_words(frequency_dict, n_words):
    freq_list = sorted(frequency_dict, key=frequency_dict.get, reverse=True)
    freq_list = list(filter(lambda x: in_model(x), freq_list))
    return freq_list[:n_words]

N_MOST_FREQUENT_WORDS = 9
freq_dict = build_freq_dict(input_data)
most_freq_words = get_most_frequent_words(freq_dict, N_MOST_FREQUENT_WORDS)
print(f'Most frequent words: {most_freq_words}')

Most frequent words: ['film', 'napoleon', 'trailer', 'care', 'therapy', 'corsican', 'shown', 'year', 'french']


### 1.2 Ottenimento degli embeddings

In [137]:
def retrieve_embeddings(words):
    res = []
    for w in words:
        res.append(word_embeddings_model[w])
    return res

embeddings = retrieve_embeddings(most_freq_words)

### 1.3 Clustering con KMeans

Qui notiamo che il numero di cluster che vogliamo ottenere da KMeans coincide al numero di paragrafi che stiamo cercando nel testo.

In [138]:
def cluster_frequent_words(embeddings, n_clust):
    model = KMeans(n_clusters=n_clust)
    model.fit(embeddings)
    return model

def extract_results(km, words):
    result = dict((label, []) for label in set(km.labels_))
    for i, label in enumerate(km.labels_):
        result[label].append(words[i])
    return result

N_SEGMENTS = 3
km = cluster_frequent_words(embeddings, N_SEGMENTS)
clusters = extract_results(km, most_freq_words)
print("Clusters are:")
pprint.pprint(clusters)

Clusters are:
{0: ['napoleon', 'corsican', 'french'],
 1: ['care', 'therapy'],
 2: ['film', 'trailer', 'shown', 'year']}


### 2.1 Classificazione per presenza

In [149]:
def occurrence_classification(input_data, clusters):
    classification_list = [-1]*len(input_data)
    for i, sent in enumerate(input_data):
        max_clust = find_max_clust(sent.split(), clusters)
        classification_list[i] = max_clust
    return classification_list

def find_max_clust(sentence, clusters):
    sent = set(to_singular(word).lower() for word in sentence)
    max_clust = -1
    max_sovr = 0
    for clust, synonyms in clusters.items():
        sovr = len(sent & set(synonyms))
        if sovr > max_sovr:
            max_clust = clust
            max_sovr = sovr
    return max_clust

### 2.2 Classificazione per prossimità

In [140]:
def proximity_classification(classification_list):
    new_list = classification_list[:]
    for sent, clust in enumerate(classification_list):
        if clust != -1:
            continue
        closest_clust = find_closest_clust(sent, classification_list)
        new_list[sent] = closest_clust
    return new_list

def find_closest_clust(position, classification_list):
    def compute_distance(start, end, step):
        for i in range(start, end, step):
            if classification_list[i] != -1:
                return i - position
        return len(classification_list)+1
  
    left_distance = compute_distance(position-1,-1,-1)
    right_distance = compute_distance(position+1,len(classification_list),1)

    if abs(right_distance) <= abs(left_distance):
        return classification_list[right_distance+position]
    else:
        return classification_list[left_distance+position]

### Risultato della classificazione

Presentiamo i risultati della classificazione fatta finora come una lista la cui lunghezza è pari al numero delle frasi e i cui elementi sono il numero del cluster di riferimento per l'argomento assegnato a un paragrafo. Quindi in sostanza numeri diversi simboleggiano i diversi paragrafi.

In [141]:
def classify_data(input_data, clusters):
    classification_list = occurrence_classification(input_data, clusters)
    classification_list = proximity_classification(classification_list)
    return classification_list

classification_list = classify_data(input_data, clusters)
print(f'Classification list: {classification_list}')

Classification list: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### 3. Estrazione dei tagli con la "derivata"

Come si può vedere dai risultati della classificazione, la lista `classification_list` ha quasi la struttura di una funzione a gradini. Per esempio, sarà costantenemnte 2 per tutte le frasi che appartengono al paragrafo 2, mentre non appena entriamo nel paragrafo 1 la funzione diventa costantemente 1. C'è però un problema: all'interno dei paragrafi abbiamo dei picchi, quindi variazioni a questo pattern che rappresentano degli errori di classificazione. Per poter estrarre i tagli è necessario distinguere questi picchi dai gradini e qui introduciamo un concetto che prende ispirazione dalla derivata delle funzioni.

In particolare, un picco si distingue da un gradino osservandone un certo intorno. Per esempio, se ho questa sequenza: `1,1,0,1,1` lo zero è chiaramente un picco, perché poi la funzione ritorna a valere 1. Tipicamente nello studio delle funzioni continue per studiare un certo intorno di un punto si utilizza la derivata, che è il limite del rapporto incrementale. In questo caso la funzione è discreta e quindi possiamo basarci solamente sulle differenze tra gli estremi di un intorno sempre più piccolo. In particolare stabiliamo una finestra di valori, la cui dimensione sarà data dall'iperparametro SPAN, e facciamo la medie delle differenze delle coppie opposte. Quindi per esempio nella lista `[1,1,0,0,0]` supponendo una finestra con span 2, la posizione 2 avrà una pseudo-derivata di $ \frac{1-0 + 1-0}{2} = 1 $. Mentre la stessa posizione nella lista `[1,1,0,1,1]` avrà una pseudo-derivata di $ \frac{1-1 + 1-1}{2} = 0 $.

Utilizzando questo meccanismo siamo in grado di distinguere facilmente i picchi dai gradini della funzione perché i primi saranno caratterizzati da pseudo-derivata < 1, mentre gli ultimi l'avranno esattamente uguale a 1.

Tutto quanto detto finora funziona solo se la lista è composta unicamente da 0 e da 1. Se introduciamo anche altri numeri l'estrazione dei picchi diventa immensamente più complicata. Pertanto `classification_list` deve essere modificata. Da notare che questa modifica causa una perdita dell'informazione, ma è informazione che non è rilevante ai fini dell'output dell'algoritmo. Infatti ci interessa unicamente individuare le posizioni dei tagli, e non stabilire l'ordine degli argomenti affrontati dal documento. Pertanto operiamo anzitutto una trasformazione della lista in termini di 0 e 1, per poi effettuare l'estrazione dei tagli utilizzando la pseudo-derivata descritta sopra.

__Iperparametro SPAN:__ la sua definizione è fondamentale per la buona estrazione dei tagli. In particolare i picchi vengono correttamente identificati solo se hanno una lunghezza inferiore alla metà dello span. Ho inserito un meccanismo di controllo che conti gli 1 della pseudo-derivata e faccia sapere all'utente se questi sono più del numero di tagli da lui specificato.

In [146]:
def derivative_based_extraction(classification_list, span, n_segments):
    paragraph_list = transform_in_paragraph(classification_list)
    derivative = build_derivative(paragraph_list, span)
    count = Counter(derivative)
    if count[1] > n_segments-1:
        print("The extraction of cuts is uncertain. Try with a larger span.")
    positions = []
    for i, el in enumerate(derivative):
        if el == 1:
            positions.append(i)
    return positions

def transform_in_paragraph(classification_list):
    result = [0]
    for i in range(1,len(classification_list)):
        if classification_list[i] == classification_list[i-1]:
            result.append(result[-1])
        else:
            result.append(int(not result[-1]))
    return result

def build_derivative(classification_list, span):
    derivative = []
    p = int(span/2)
    for i in range(p,len(classification_list)-p):
        window = classification_list[i-p:i+p]
        diff = []
        for j in range(p):
            diff.append(window[j]-window[-(j+1)])
        derivative.append(abs(mean(diff)))
    return derivative

In [147]:
SPAN = 10
cuts = derivative_based_extraction(classification_list, SPAN, N_SEGMENTS)
print(f'Cuts found by the algorithm: {cuts}')
print(f'Correct cuts: {correct_cuts}')

Cuts found by the algorithm: [25, 43]
Correct cuts: [30, 47]
